<a href="https://colab.research.google.com/github/ge96lip/Quantum-Computing/blob/main/QC_Shor's_Factorization_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

In [58]:
!pip install qiskit
!pip install pylatexenc
!pip install qiskit_aer

In [59]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Qubit
from qiskit.circuit.library import XGate, CSwapGate, QFT
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

import matplotlib.pyplot as plt
import math
%matplotlib inline

# General Functions

**Here are some values of N to try:**

15, 21, 33, 35, 39, 51, 55, 69, 77, 85, 87, 91, 93, 95, 111, 115, 117,
119, 123, 133, 155, 187, 203, 221, 247, 259, 287, 341, 451

**Larger numbers require more bits of precision.**

N = 15    precision_bits >= 4

N = 21    precision_bits >= 5

N = 35    precision_bits >= 6

N = 123   precision_bits >= 7

N = 341   precision_bits >= 8  time: about 6 seconds

N = 451   precision_bits >= 9  time: about 23 seconds

In [60]:
def shor_sample():
    N = 35             # The number we're factoring
    precision_bits = 4
    coprime = 2

    result = Shor(N, precision_bits, coprime)

    if result != None:
        print('Success! '+str(N)+'='+str(result[0])+'*'+str(result[1])+'\n');
    else:
        print('Failure: No non-trivial factors were found.\n')

In [61]:
def ShorLogic(N, repeat_period_candidates, coprime):
    print('Repeat period candidates: '+str(repeat_period_candidates)+'\n')
    factor_candidates = []
    for i in range(len(repeat_period_candidates)):
        repeat_period = repeat_period_candidates[i];
        # Given the repeat period, find the actual factors
        ar2 = pow(coprime, repeat_period / 2.0)
        factor1 = int(gcd(N, ar2 - 1))
        factor2 = int(gcd(N, ar2 + 1))
        factor_candidates.append([factor1, factor2])
    return factor_candidates

In [62]:
def gcd(a, b):
    # return the greatest common divisor of a,b
    while b:
        m = a % b
        a = b
        b = m
    return a

In [63]:
def check_result(N, factor_candidates):
    for i in range(len(factor_candidates)):
        factors = factor_candidates[i]
        if factors[0] * factors[1] == N:
            if factors[0] != 1 and factors[1] != 1:
                # Success!
                return factors
    # Failure
    return None

# Classical Factorization Algorithm

In [64]:
def Shor(N, precision_bits, coprime):
    repeat_period = ShorNoQPU(N, precision_bits, coprime) # quantum part
    factors = ShorLogic(N, repeat_period, coprime)        # classical part
    return check_result(N, factors)

In [65]:
def ShorNoQPU(N, precision_bits, coprime):
    # Classical replacement for the quantum part of Shor
    repeat_period_candidates = []
    work = 1
    max_loops = pow(2, precision_bits)
    for iter in range(max_loops):
        work = (work * coprime) % N
        if work == 1: # found a repeat period
            repeat_period_candidates.append(iter + 1)
    return repeat_period_candidates

In [66]:
shor_sample()

Repeat period candidates: [12]

Success! 35=7*5



# Shor's Factorization Algorithm

In [67]:
def Shor(N, precision_bits, coprime):
    repeat_period = ShorQPU(N, precision_bits, coprime) # quantum part
    factors = ShorLogic(N, repeat_period, coprime)      # classical part
    return check_result(N, factors)

In [68]:
def ShorQPU(N, precision_bits, coprime):
    # Quantum part of Shor's algorithm
    # For this implementation, the coprime must be 2.
    coprime = 2;

    # For some numbers (like 15 and 21) the "mod" in a^xmod(N)
    # is not needed, because a^x wraps neatly around. This makes the
    # code simpler, and much easier to follow.
    if N == 15 or N == 21:
        return ShorQPU_WithoutModulo(N, precision_bits, coprime)
    else:
        return ShorQPU_WithModulo(N, precision_bits, coprime)

In [69]:
def RollLeft(qc, work, num_shifts, control):
    # Ensure we're not out of range for work qubits
    for j in range(work.size - num_shifts-1,-1,-1):
        # Apply CSWAP between work[j] <-> work[j + num_shifts]
        qc.append(CSwapGate(), [control, work[j], work[j + num_shifts]])
    return qc

In [70]:
# In case our QPU read returns a "signed" negative value,
# convert it to unsigned.
def read_unsigned(qc, qreg, creg):
    qc.measure(qreg, creg) #value = qreg.read()

    # Simulate the circuit using the Aer simulator
    simulator = AerSimulator()
    job = simulator.run(transpile(qc, simulator), shots=1)#1024)
    result = job.result()
    counts = result.get_counts(qc)

    # Visualize the results
    print("Measurement results: " + str(counts) + "\n")
    fig = plot_histogram(counts)
    fig.show()

    # Extract value of most likely measurement outcome
    value = int(max(counts, key=counts.get),2)

    return value & ((1 << qreg.size) - 1)

In [71]:
def estimate_num_spikes(spike, spike_range):
    if spike < spike_range / 2:
        spike = spike_range - spike
    best_error = 1.0
    e0 = 0
    e1 = 0
    e2 = 0
    actual = spike / spike_range
    candidates = []
    for denominator in range(1,spike):
        numerator = round(denominator * actual)
        estimated = numerator / denominator
        error = abs(estimated - actual)
        e0 = e1
        e1 = e2
        e2 = error
        # Look for a local minimum which beats our
        # current best error
        if e1 <= best_error or e1 < e0 or e1 < e2:
            repeat_period = denominator - 1
            candidates.append(repeat_period)
            best_error = e1
    return candidates

In [72]:
# This is the short/simple version of ShorQPU() where we can perform a^x and
# don't need to be concerned with performing a quantum int modulus.

def ShorQPU_WithoutModulo(N, precision_bits, coprime):
    N_bits = 1
    while (1 << N_bits) < N:
        N_bits +=1
    if N != 15: # For this implementation, numbers other than 15 need an extra bit
        N_bits +=1

    # Set up the QPU and the working registers
    work = QuantumRegister(N_bits, name="work")
    precision = QuantumRegister(precision_bits, name="precision")
    classical = ClassicalRegister(precision_bits, name="precision_measure")
    qc = QuantumCircuit(work, precision, classical)

    # Initialization: set the working register to state |1>
    # and the precision register to state |0>
    # put the precision register into superposition using Hadamard gate
    qc.initialize(1, work)
    qc.initialize(0, precision)
    qc.h(precision)

    qc.barrier()

    # Perform 2^x for all possible values of x in superposition
    for iter in range(precision_bits):
        num_shifts = 1 << iter
        if num_shifts < N_bits:
            qc = RollLeft(qc, work, num_shifts, precision[iter])

    qc.barrier()

    # Quantum Fourier Transform on precision register
    qc.append(QFT(precision_bits), precision)

    qc.barrier()

    print(qc)

    read_result = read_unsigned(qc, precision, classical)
    print('QPU read result: '+str(read_result)+'\n')
    repeat_period_candidates = estimate_num_spikes(read_result, 1 << precision_bits)

    return repeat_period_candidates

In [120]:
################################################################################
# This function is not working yet ! ###########################################
################################################################################

# This is the complicated version of ShorQPU() where we DO
# need to be concerned with performing a quantum int modulus.
# That's a complicated operation, and it also requires us to
# do the shifts one at a time.
def ShorQPU_WithModulo(N, precision_bits, coprime):
    scratch = None
    max_value = 1
    mod_engaged = False

    first = True

    N_bits = 1
    scratch_bits = 0
    while (1 << N_bits) < N:
        N_bits +=1
    if N != 15: # For this implementation, numbers other than 15 need an extra bit
        N_bits +=1
    scratch_bits = 1

    # Set up the QPU and the working registers
    work = QuantumRegister(N_bits, name="work")
    precision = QuantumRegister(precision_bits, name="precision")
    scratch = QuantumRegister(1, name="scratch")
    classical = ClassicalRegister(precision_bits, name="precision_measure")
    qc = QuantumCircuit(work, precision, scratch, classical)

    # Initialization: set the working register to state |1>
    # and the precision register to state |0>
    # put the precision register into superposition using Hadamard gate
    qc.initialize(1, work)
    qc.initialize(0, precision)
    qc.initialize(0, scratch)
    qc.h(precision)

    qc.barrier()

    #qc = add_int(qc, work, scratch, -N, condition=None)

    #qc.barrier()
    #N_sign_bit_place = 1 << (N_bits - 1)
    #N_sign_bit = num.bits(N_sign_bit_place)
    N_sign_bit_place = N_bits - 1
    N_sign_bit = work[N_sign_bit_place]

    #print(N_sign_bit_place)
    #print(N_sign_bit)

    for iter in range(precision_bits):
        condition = precision[iter]  # Get the condition qubit
        N_sign_bit_with_condition = N_sign_bit or condition

        #print(N_sign_bit_with_condition)
        # Create condition logic (or gate)
        #qc.x(condition)  # Use NOT for controlled logic (X gate)
        #qc.ccx(condition, N_sign_bit_with_condition, N_sign_bit_with_condition)

        #condition = precision.bits(1 << iter)
        #N_sign_bit_with_condition = num.bits(N_sign_bit_place)
        #N_sign_bit_with_condition.orEquals(condition)

        shifts = 1 << iter
        for shift in range(shifts):
            #qc = RollLeft(qc, work, 1, precision[iter]) # multiply by the coprime 2
            max_value <<= 1

            if max_value >= N:
                mod_engaged = True

            if mod_engaged:

                # Perform conditional subtraction and check for wrap
                #qc = append_modulo_N(qc, N, work, scratch, precision[iter])
                #qc.not_(work[0])
                #qc.cx(work[0], scratch[0])  # Controlled-NOT based on the sign bit
                #qc.not_(work[0])


                if first:
                    #print(condition)
                    wrap_mask = scratch
                    wrap_mask_with_condition = wrap_mask or condition

                    #qc = substract(qc, work, N, condition)
                    qc = add_int(qc, work, -N, condition) # Substract N from working register, causing this to go negative if work < N
                    #qc.cx(N_sign_bit_with_condition)
                    first = False

                '''
                # Subtract N, which may cause negative value
                qc.x(work[0])  # Negate the first bit
                qc.cx(work[0], wrap_mask)  # Controlled subtraction

                # Skim off the sign bit
                qc.cnot(N_sign_bit_with_condition, wrap_mask)  # Controlled NOT

                # If went negative, undo the subtraction
                qc.x(work[0])  # Undo negation
                qc.cx(wrap_mask_with_condition, work[0])  # Conditional addition

                # Placeholder for clearing wrap bit if odd
                qc.x(work[0])  # Placeholder for condition clearing
                '''

                '''
                wrap_mask = scratch.bits()
                wrap_mask_with_condition = scratch.bits()
                wrap_mask_with_condition.orEquals(condition)

                # Here's the modulo code.
                num.subtract(N, condition) # subtract N, causing this to go negative if we HAVEN'T wrapped.
                scratch.cnot(N_sign_bit_with_condition) # Skim off the sign bit
                num.add(N, wrap_mask_with_condition) # If we went negative, undo the subtraction.
                num.not(1)
                scratch.cnot(num, 1, condition) # If it's odd, then we wrapped, so clear the wrap bit
                num.not(1)
                '''

    # Quantum Fourier Transform on precision register
    #qc.append(QFT(precision_bits), precision)

    print(qc)

    read_result = read_unsigned(qc, precision, classical)
    print('QPU read result: '+str(read_result)+'\n')
    repeat_period_candidates = estimate_num_spikes(read_result, 1 << precision_bits)



    return repeat_period_candidates

In [180]:
shor_sample()

condition mask  64
cmask  64
add val mask  1
OPS: cmask  0b1000000
OPS: ~add_shift_mask  -0b1000001
OPS: add shift mask  64
OPS: cmask  0b0
OPS: ~add_shift_mask  -0b100001
OPS: add shift mask  32
OPS: cmask  0b0
OPS: ~add_shift_mask  -0b10001
OPS: add shift mask  16
OPS: cmask  0b0
OPS: ~add_shift_mask  -0b1001
OPS: add shift mask  8
OPS: cmask  0b0
OPS: ~add_shift_mask  -0b101
OPS: add shift mask  4
OPS: cmask  0b0
OPS: ~add_shift_mask  -0b11
OPS: add shift mask  2
OPS: cmask  0b0
OPS: ~add_shift_mask  -0b10
OPS: add shift mask  1
ops  [(64, 0), (32, 0), (16, 0), (8, 0), (4, 0), (2, 0), (1, 0)]
cmask  64
add val mask  2
OPS: cmask  0b1000000
OPS: ~add_shift_mask  -0b1000001
OPS: add shift mask  64
OPS: cmask  0b0
OPS: ~add_shift_mask  -0b100001
OPS: add shift mask  32
OPS: cmask  0b0
OPS: ~add_shift_mask  -0b10001
OPS: add shift mask  16
OPS: cmask  0b0
OPS: ~add_shift_mask  -0b1001
OPS: add shift mask  8
OPS: cmask  0b0
OPS: ~add_shift_mask  -0b101
OPS: add shift mask  4
OPS: cmask  

In [179]:
def add_int(qc, qreg, N, condition):
    reverse_to_subtract = False
    if N == 0:
        return
    elif N < 0:
        N = -N
        reverse_to_subtract = True
    ops = []
    add_val = int(N)
    condition_mask = 1 << (len(qreg) - 1)
    #condition_mask = (len(qreg) - 1)

    print("condition mask ", condition_mask)

    add_val_mask = 1
    while add_val_mask <= add_val:
        cmask = condition_mask & ~(add_val_mask - 1)
        print("cmask ", cmask)
        if add_val_mask & add_val:
            print("add val mask ", add_val_mask)
            add_shift_mask = 1 << (len(qreg) - 1)
            while add_shift_mask >= add_val_mask:
                print("OPS: cmask ", str(bin(cmask)))
                print("OPS: ~add_shift_mask ", str(bin(~add_shift_mask)))
                cmask &= ~add_shift_mask
                #cmask = cmask & ~(add_shift_mask - 1)
                print("OPS: add shift mask ", add_shift_mask)

                ops.append((add_shift_mask, cmask))
                add_shift_mask >>= 1
            print("ops ", ops)
        condition_mask &= ~add_val_mask
        add_val_mask <<= 1
    if reverse_to_subtract:
        ops.reverse()
    for inst in ops:
        print("inst ",inst)
        op_qubits = []
        #mask = 1
        for i in range(len(qreg)):
            #print("inst[1] ", inst[1])
            #print("1<<i ", 1 << i)
            if inst[1] & (1 << i):
                #print("append")
                op_qubits.append(qreg[i])
        for i in range(len(qreg)):
            #print("inst[0] ", inst[0])
            #print("1<<i ", 1 << i)
            if inst[0] & (1 << i):
                #print("append")
                op_qubits.append(qreg[i])
        qc = multi_cx(qc, op_qubits, condition)

    return qc


def multi_cx(qc, qubits, condition=None, do_cz=False):
    ## This will perform a CCCCCX with as many conditions as we want
    ## The last qubit in the list is the target.

    target = qubits[-1]
    conds = qubits[:-1]
    ops = []

    print("target ", target)
    print("conds ", conds)

    controll_qubits = []

    for i in range(len(conds)):
        controll_qubits.append(conds[i])
    if condition != None:
        controll_qubits.append(condition)
    print("controll qubits ", controll_qubits)
    #qc.mcx([controll_qubits], target)

    if len(conds) == 0:
        #qc.x(target)
        qc.cx(condition,target)
    elif len(conds) == 1:
        #qc.cx(conds[0], target)
        qc.ccx(condition, conds[0], target)
    else:
        #qc.ccx(conds[0], conds[1], target)
        #qc.c3x(condition, conds[0], conds[1], target)
        qc.mcx([condition, conds[0], conds[1]], target)
    #if do_cz:
    #    qc.h(target)
    #ops.reverse()
    #for op in ops:
        #qc.ccx(op[0], op[1], op[2])
        #qc.c3x(condition, op[0], op[1], op[2])
    #    qc.mcx([condition, op[0], op[1]], op[2])

    return qc

In [76]:



def main():

    # Set up the program
    work = QuantumRegister(5, name='work')
    classical = ClassicalRegister(5, name='classical')
    precision = QuantumRegister(4, name='precision')
    scratch = QuantumRegister(1, name='scratch')
    qc = QuantumCircuit(work, precision, scratch, classical)
    qc.initialize(20, work)
    qc.initialize(0, precision)
    qc.initialize(0, scratch)
    qc.h(precision)

    ## initialization
    #qc.x(work[0])
    #qc.h(work[2])
    #qc.rz(math.radians(45), work[2])
    qc.barrier()

    ## Increment
    qc = add_int(qc, work, scratch, -15)

    qc.barrier()
    ## Decrement
    #add_int(a, -1)
    print(qc)

    qc.measure(work, classical)
    #qc.draw()
    # Run the circuit
    simulator = AerSimulator()
    job = simulator.run(transpile(qc, simulator), shots=1024)
    result = job.result()
    counts = result.get_counts(qc)

    # Display the circuit and output
    #print(qc)
    plot_histogram(counts)




main()




TypeError: add_int() missing 1 required positional argument: 'condition'

In [ ]:

from qiskit.visualization import plot_histogram
import numpy as np


def modular_addition_circuit(qc, x_qreg, N, scratch):
    # Initialize a quantum circuit with the necessary qubits and classical registers
    #x_qreg = QuantumRegister(num_qubits, 'x')  # Quantum register for 'x'
    #ancilla = QuantumRegister(1, 'ancilla')  # Ancilla qubit for control
    #classical_reg = ClassicalRegister(num_qubits, 'result')  # Classical register to store measurement results
    #qc = QuantumCircuit(x_qreg, ancilla, classical_reg)
    num_qubits = x_qreg.size
    # Load the value of x in binary form into the quantum register 'x_qreg'
    x_binary = format(x, f'0{num_qubits}b')
    for i, bit in enumerate(reversed(x_binary)):
        if bit == '1':
            qc.x(x_qreg[i])  # Apply X gate to set qubit if the corresponding bit in x is 1

    # Modular addition to compute x mod N
    # We implement x mod N by flipping the ancilla qubit when x >= N and using it to control "subtraction"
    N_binary = format(N, f'0{num_qubits}b')
    for i, bit in enumerate(reversed(N_binary)):
        if bit == '1':
            qc.cx(x_qreg[i], scratch[0])

    # If ancilla is flipped, subtract N from x to get x mod N
    for i, bit in enumerate(reversed(N_binary)):
        if bit == '1':
            qc.cx(scratch[0], x_qreg[i])

    # Measure the result in the classical register
    #qc.measure(x_qreg, classical_reg)

    return qc

# Parameters
x = 5   # Example value of x
N = 3   # Example modulus N
num_qubits = max(len(bin(x)[2:]), len(bin(N)[2:]))  # Determine the number of qubits needed

# Build and simulate the circuit
qc = modular_addition_circuit(x, N, num_qubits)
qc.draw('mpl')

# Run the circuit
simulator = AerSimulator()
job = simulator.run(transpile(qc, simulator), shots=1024)
result = job.result()
counts = result.get_counts(qc)

# Display the circuit and output
print(qc)
plot_histogram(counts)


In [ ]:
################################################################################
# This function is not working yet ! ###########################################
################################################################################

def append_modulo_N(qc, N, work, scratch, condition=None):
    # Performs a controlled modular reduction on the 'work' register with
    # respect to N, using the 'scratch' register to store the sign bit.

    N_bits = work.size
    N_binary = format(N, 'b').zfill(N_bits)  # Binary representation of N with padding to match work size
    print(N_binary)
    # Step 1: Conditionally subtract N (controlled by `condition` qubit)
    # We use multi-controlled X gates to simulate the effect of conditional subtraction.
    for i, bit in enumerate(reversed(N_binary)):
        if bit == '1':  # Only operate on bits where N has a '1'
            if condition:
                # Apply controlled X gate if the condition qubit is set
                qc.ccx(condition, work[i], scratch[0])  # Controlled-controlled-X for the i-th bit
            else:
                qc.cx(work[i], scratch[0])

    # Step 2: Use the `scratch` qubit to store the sign bit
    # Copy the most significant bit (MSB) of `work` to `scratch` to store the sign
    qc.cx(work[-1], scratch[0])

    # Step 3: Conditionally add N back if the sign bit (scratch) indicates overflow
    # This is done if the scratch bit is set

    # Use an auxiliary qubit to handle the addition
    ancilla = QuantumRegister(1, 'ancilla')  # Create an ancilla qubit for addition
    qc.add_register(ancilla)  # Add the ancilla register to the circuit
    for i, bit in enumerate(reversed(N_binary)):
        if bit == '1':  # Add N back for each '1' bit in N's binary representation
            # Apply addition based on the `scratch` bit as the control
            #qc.cx(scratch[0], work[i])  # Controlled addition bitwise

            # Prepare the ancilla to perform addition if needed
            qc.x(ancilla[0])  # Initialize ancilla qubit to 1 for addition

            # Controlled addition using the scratch qubit as control
            qc.ccx(scratch[0], ancilla[0], work[i])  # Add N back if overflow occurred
            qc.ccx(scratch[0], work[i], ancilla[0])  # This operation ensures the original value remains

            # Cleanup: if the scratch bit is zero, we need to undo the addition from the ancilla
            qc.cx(ancilla[0], work[i])  # Conditional copy from ancilla to work
            qc.measure(ancilla, classical[0])  # Measure the ancilla for debugging if needed


    # Step 4: Clear the wrap bit if necessary
    # Here we conditionally reset `scratch` to 0 if `work` is odd.
    qc.cx(work[0], scratch[0])  # Clear scratch based on the least significant bit (LSB) of `work`

    return qc

In [ ]:
N = 3  # Modulus
work_bits = 3  # Number of qubits in the work register
scratch_bits = 1  # Scratch qubit to store the overflow or sign bit

# Create quantum and classical registers
work = QuantumRegister(work_bits, 'work')
scratch = QuantumRegister(scratch_bits, 'scratch')
classical = ClassicalRegister(work_bits, 'classical')

# Create quantum circuit
qc = QuantumCircuit(work, scratch, classical)

# Prepare the `work` register in a state that exceeds N
# For example, set the state to |5> (binary: 101) which should reduce to 2 (5 % 3 = 2)
qc.x(work[0])  # Set qubit 0 to 1
qc.x(work[2])  # Set qubit 2 to 1, representing |5>

qc.measure(work, classical)
print(qc)

# Apply the modulo function
qc = append_modulo_N(qc, N, work, scratch)

print(qc)

# Measure the work register to see the result
qc.measure(work, classical)

# Execute the circuit on the Aer simulator
simulator = AerSimulator()
job = simulator.run(transpile(qc, simulator), shots=1)#1024)
result = job.result()

# Get and plot the results
counts = result.get_counts(qc)
print("Measurement results:", counts)
plot_histogram(counts)
plt.show()